In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import os
import random
from torch.autograd import Variable
import copy
from torch import nn, optim
import torch.optim as optim
import numpy as np
from torch.autograd import Variable
from collections import OrderedDict
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import csv
import time
import math


In [2]:
def fix_seed(seed):
    # random
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

SEED = 42
fix_seed(SEED)

In [3]:
class Argments():
  def __init__(self):
    self.batch_size = 10
    self.test_batch = 1000
    self.global_epochs = 100
    self.local_epochs = 2
    self.lamda = 15
    self.K = 5
    self.lr = None
    self.momentum = 0.9
    self.weight_decay = 10**-4.0
    self.clip = 20.0
    self.partience = 100
    self.worker_num = 20
    self.sample_num = 20
    self.device = device = torch.device('cuda:0'if torch.cuda.is_available() else'cpu')
    self.criterion = nn.CrossEntropyLoss()

args = Argments()

In [4]:
lr = 0

In [6]:
lr_list = []
lr_list.append(10**-3.0)
lr_list.append(10**-2.5)
lr_list.append(10**-2.0)
lr_list.append(10**-1.5)
lr_list.append(10**-1.0)
lr_list.append(10**-0.5)
lr_list.append(10**0.0)
lr_list.append(10**0.5)

args.lr = lr_list[lr]

In [9]:
class LocalDataset(torch.utils.data.Dataset):
  def __init__(self,dataset,worker_id):
    self.data = []
    self.target = []
    self.id = worker_id
    for i in range(len(dataset)):
      self.data.append(dataset[i][0][0])
      self.target.append(dataset[i][1][0])

  def __getitem__(self, index):
    return self.data[index],self.target[index]

  def __len__(self):
    return len(self.data)

In [10]:
with open('../data/federated_trainset_shakespeare.pickle', 'rb') as f:
    all_federated_trainset = pickle.load(f)
with open('../data/federated_testset_shakespeare.pickle', 'rb') as f:
    all_federated_testset = pickle.load(f)
all_worker_num = len(all_federated_trainset)

In [11]:
worker_id_list = random.sample(range(all_worker_num),args.worker_num)
print(worker_id_list)
federated_trainset = []
federated_testset = []
for i in worker_id_list:
    federated_trainset.append(all_federated_trainset[i])
    federated_testset.append(all_federated_testset[i])

[28, 6, 70, 62, 57, 35, 26, 139, 22, 108, 8, 7, 23, 55, 59, 129, 50, 107, 56, 114]


In [12]:
federated_valset = [None]*args.worker_num
for i in range(args.worker_num):
  n_samples = len(federated_trainset[i])
  if n_samples==1:
    federated_valset[i] = copy.deepcopy(federated_trainset[i])
  else:
    train_size = int(len(federated_trainset[i]) * 0.7) 
    val_size = n_samples - train_size 
    federated_trainset[i],federated_valset[i] = torch.utils.data.random_split(federated_trainset[i], [train_size, val_size])

In [14]:
class RNN1(nn.Module):
    def __init__(self, embedding_dim=8, vocab_size=90, hidden_size=256):
        super(RNN1, self).__init__()
        self.embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=1, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, input_seq):
        embeds = self.embeddings(input_seq)
        # Note that the order of mini-batch is random so there is no hidden relationship among batches.
        # So we do not input the previous batch's hidden state,
        # leaving the first hidden state zero `self.lstm(embeds, None)`.
        lstm_out, _ = self.lstm(embeds)
        # use the final hidden state as the next character prediction
        final_hidden_state = lstm_out[:, -1]
        output = self.fc(final_hidden_state)
        return output

class RNN2(nn.Module):
    def __init__(self, embedding_dim=8, vocab_size=90, hidden_size=256):
        super(RNN2, self).__init__()
        self.embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=2, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, input_seq):
        embeds = self.embeddings(input_seq)
        # Note that the order of mini-batch is random so there is no hidden relationship among batches.
        # So we do not input the previous batch's hidden state,
        # leaving the first hidden state zero `self.lstm(embeds, None)`.
        lstm_out, _ = self.lstm(embeds)
        # use the final hidden state as the next character prediction
        final_hidden_state = lstm_out[:, -1]
        output = self.fc(final_hidden_state)
        return output

class RNN3(nn.Module):
    def __init__(self, embedding_dim=8, vocab_size=90, hidden_size=256):
        super(RNN3, self).__init__()
        self.embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=3, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, input_seq):
        embeds = self.embeddings(input_seq)
        # Note that the order of mini-batch is random so there is no hidden relationship among batches.
        # So we do not input the previous batch's hidden state,
        # leaving the first hidden state zero `self.lstm(embeds, None)`.
        lstm_out, _ = self.lstm(embeds)
        # use the final hidden state as the next character prediction
        final_hidden_state = lstm_out[:, -1]
        output = self.fc(final_hidden_state)
        return output
    
class RNN4(nn.Module):
    def __init__(self, embedding_dim=8, vocab_size=90, hidden_size=256):
        super(RNN4, self).__init__()
        self.embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=4, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, input_seq):
        embeds = self.embeddings(input_seq)
        # Note that the order of mini-batch is random so there is no hidden relationship among batches.
        # So we do not input the previous batch's hidden state,
        # leaving the first hidden state zero `self.lstm(embeds, None)`.
        lstm_out, _ = self.lstm(embeds)
        # use the final hidden state as the next character prediction
        final_hidden_state = lstm_out[:, -1]
        output = self.fc(final_hidden_state)
        return output

In [15]:
class pFedMeOptimizer(torch.optim.Optimizer):
    def __init__(self, params, lr=0.01, lamda=0.1 , mu = 0.001):
        #self.local_weight_updated = local_weight # w_i,K
        if lr < 0.0:
            raise ValueError("Invalid learning rate: {}".format(lr))
        defaults = dict(lr=lr, lamda=lamda, mu = mu)
        super(pFedMeOptimizer, self).__init__(params, defaults)
    
    def step(self, local_weight_updated, closure=None):
        loss = None
        if closure is not None:
            loss = closure
        weight_update = local_weight_updated.copy()
        for group in self.param_groups:
            for p, localweight in zip( group['params'], weight_update):
                localweight.data = localweight.data.to(args.device)
                p.data = p.data - group['lr'] * (p.grad.data + group['lamda'] * (p.data - localweight.data) + group['mu']*p.data)
                localweight.data = localweight.data.to('cpu')
        return  group['params'], loss
    
    def update_param(self, local_weight_updated, closure=None):
        loss = None
        if closure is not None:
            loss = closure
        weight_update = local_weight_updated.copy()
        for group in self.param_groups:
            for p, localweight in zip( group['params'], weight_update):
                p.data = localweight.data
        #return  p.data
        return  group['params']

In [17]:
class Server():
  def __init__(self):
    self.model = RNN2()

  def create_worker(self,federated_trainset,federated_valset,federated_testset):
    workers = []
    for i in range(args.worker_num):
      workers.append(Worker(federated_trainset[i],federated_valset[i],federated_testset[i]))
    return workers

  def sample_worker(self,workers):
    sample_worker = []
    sample_worker_num = random.sample(range(args.worker_num),args.sample_num)
    for i in sample_worker_num:
      sample_worker.append(workers[i])
    return sample_worker


  def send_model(self,workers):
    nums = 0
    for worker in workers:
      nums += worker.train_data_num

    for worker in workers:
      worker.aggregation_weight = 1.0*worker.train_data_num/nums
      worker.model = copy.deepcopy(self.model)
      worker.personalized_model = copy.deepcopy(self.model)
      worker.local_model = copy.deepcopy(self.model)

  def aggregate_model(self,workers):   
    new_params = OrderedDict()
    for i,worker in enumerate(workers):
      worker_state = worker.model.state_dict()
      for key in worker_state.keys():
        if i==0:
          new_params[key] = worker_state[key]*worker.aggregation_weight
        else:
          new_params[key] += worker_state[key]*worker.aggregation_weight
    self.model.load_state_dict(new_params)
    
  def send_parameters(self,workers):
    nums = 0
    for worker in workers:
      nums += worker.train_data_num
    for worker in workers:
        worker.aggregation_weight = 1.0*worker.train_data_num/nums
        worker.set_parameters(self.model)

In [18]:
class Worker():
  def __init__(self,trainset,valset,testset):
    self.trainloader = torch.utils.data.DataLoader(trainset,batch_size=args.batch_size,shuffle=True,num_workers=2)
    self.valloader = torch.utils.data.DataLoader(valset,batch_size=args.test_batch,shuffle=False,num_workers=2)
    self.testloader = torch.utils.data.DataLoader(testset,batch_size=args.test_batch,shuffle=False,num_workers=2)
    #self.iter_trainloader = iter(self.trainloader)
    self.model = RNN2()
    self.local_model = copy.deepcopy(list(self.model.parameters()))
    self.persionalized_model = copy.deepcopy(list(self.model.parameters()))
    self.persionalized_model_bar = copy.deepcopy(list(self.model.parameters()))
    self.train_data_num = len(trainset)
    self.test_data_num = len(testset)
    
    self.optimizer = pFedMeOptimizer(self.model.parameters(),lr=args.lr,lamda=args.lamda)
    
  def set_parameters(self, model):
    for old_param, new_param, local_param in zip(self.model.parameters(), model.parameters(), self.local_model):
        old_param.data = new_param.data.clone()
        local_param.data = new_param.data.clone()
    #self.local_weight_updated = copy.deepcopy(self.optimizer.param_groups[0]['params'])
    
  def get_next_train_batch(self):
    try:
        # Samples a new batch for persionalizing
        (X, y) = next(self.iter_trainloader)
    except StopIteration:
        # restart the generator if the previous generator is exhausted.
        self.iter_trainloader = iter(self.trainloader)
        (X, y) = next(self.iter_trainloader)
    return (X.to(args.device), y.to(args.device))    
  '''
  def local_train(self):
    self.model.train()
    optimizer = pFedMeOptimizer(self.model.parameters(),lr=args.lr,lamda=args.lamda)
    for epoch in range(args.local_epochs):
        running_loss = 0.0
        correct = 0
        count = 0
        data,labels = self.get_next_train_batch()
        for i in range(args.K):
            self.model = self.model.to(args.device)
            optimizer.zero_grad()
            outputs = self.model(data)
            loss = args.criterion(outputs,labels)
            running_loss += loss.item()
            predicted = torch.argmax(outputs,dim=1)
            correct += (predicted==labels).sum().item()
            count += len(labels)
            loss.backward()
            #torch.nn.utils.clip_grad_norm_(self.model.parameters(), args.clip)
            self.model = self.model.to('cpu')
            personal_model,_ = optimizer.step(self.local_model)
        
        for new_param,localweight in zip(personal_model,self.local_model.parameters()):
            localweight.data = localweight.data - args.lamda * args.lr * (localweight.data - new_param.data)    
    
    del self.model
    
    update_parameters(self.personalized_model,personal_model)
    return 100.0*correct/count,running_loss/len(self.trainloader)
  '''

  def local_train(self):
    self.model.train() 
    self.model = self.model.to(args.device)       
    for epoch in range(args.local_epochs):
        running_loss = 0.0
        correct = 0
        count = 0
        for (data,labels) in self.trainloader:
            self.model.train()
            data,labels = Variable(data),Variable(labels)
            data,labels = data.to(args.device),labels.to(args.device)
            for k in range(args.K):
                self.optimizer.zero_grad()
                outputs = self.model(data)
                loss = args.criterion(outputs,labels)
                if k==(args.K-1):
                    running_loss += loss.item()
                    predicted = torch.argmax(outputs,dim=1)
                    correct += (predicted==labels).sum().item()
                    count += len(labels)
                loss.backward()
                #torch.nn.utils.clip_grad_norm_(self.model.parameters(), args.clip)
                self.persionalized_model_bar,_ = self.optimizer.step(self.local_model)
        
            for new_param, localweight in zip(self.persionalized_model_bar, self.local_model):
                localweight.data = localweight.data.to(args.device)
                localweight.data = localweight.data - args.lamda* args.lr * (localweight.data - new_param.data)
                localweight.data = localweight.data.to('cpu')
    
    self.update_parameters(self.local_model)
    
    self.model = self.model.to('cpu')
    for personal_weight, localweight in zip(self.persionalized_model_bar, self.local_model):
        personal_weight.data = personal_weight.data.to('cpu')
        localweight.data = localweight.data.to('cpu')
        
    return 100.0*correct/count,running_loss/len(self.trainloader)


  def validate(self):
    self.model.eval()
    self.update_parameters(self.persionalized_model_bar)
    self.model = self.model.to(args.device)
    acc,loss = test(self.model,args.criterion,self.valloader)
    self.model = self.model.to('cpu')
    self.update_parameters(self.local_model)
    return acc,loss


  def test(self):
    self.model.eval()
    self.update_parameters(self.persionalized_model_bar)
    self.model = self.model.to(args.device)
    acc,loss = test(self.model,args.criterion,self.testloader)
    self.model = self.model.to('cpu')
    self.update_parameters(self.local_model)
    return acc,loss


  def update_parameters(self, new_params):
    for param , new_param in zip(self.model.parameters(), new_params):
      param.data = new_param.data.clone()

In [19]:
def train(model,criterion,trainloader,epochs):
  optimizer = optim.SGD(model.parameters(),lr=args.lr,momentum=args.momentum,weight_decay=args.weight_decay)
  model.train()
  for epoch in range(epochs):
    running_loss = 0.0
    correct = 0
    count = 0
    for (data,labels) in trainloader:
      data,labels = Variable(data),Variable(labels)
      data,labels = data.to(args.device),labels.to(args.device)
      optimizer.zero_grad()
      outputs = model(data)
      loss = criterion(outputs,labels)
      running_loss += loss.item()
      predicted = torch.argmax(outputs,dim=1)
      correct += (predicted==labels).sum().item()
      count += len(labels)
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
      optimizer.step()

  return 100.0*correct/count,running_loss/len(trainloader)



In [20]:
def test(model,criterion,testloader):
  model.eval()
  running_loss = 0.0
  correct = 0
  count = 0
  for (data,labels) in testloader:
    data,labels = data.to(args.device),labels.to(args.device)
    outputs = model(data)
    running_loss += criterion(outputs,labels).item()
    predicted = torch.argmax(outputs,dim=1)
    correct += (predicted==labels).sum().item()
    count += len(labels)

  accuracy = 100.0*correct/count
  loss = running_loss/len(testloader)


  return accuracy,loss

In [21]:
class Early_Stopping():
  def __init__(self,partience):
    self.step = 0
    self.loss = float('inf')
    self.partience = partience

  def validate(self,loss):
    if self.loss<loss:
      self.step += 1
      if self.step>self.partience:
        return True
    else:
      self.step = 0
      self.loss = loss

    return False

In [22]:
server = Server()
workers = server.create_worker(federated_trainset,federated_valset,federated_testset)
acc_train = []
loss_train = []
acc_valid = []
loss_valid = []

early_stopping = Early_Stopping(args.partience)

start = time.time()

for epoch in range(args.global_epochs):
  sample_worker = server.sample_worker(workers)
  server.send_parameters(sample_worker)

  acc_train_avg = 0.0
  loss_train_avg = 0.0
  acc_valid_avg = 0.0
  loss_valid_avg = 0.0
  for worker in sample_worker:
    acc_train_tmp,loss_train_tmp = worker.local_train()
    acc_valid_tmp,loss_valid_tmp = worker.validate()
    acc_train_avg += acc_train_tmp/len(sample_worker)
    loss_train_avg += loss_train_tmp/len(sample_worker)
    acc_valid_avg += acc_valid_tmp/len(sample_worker)
    loss_valid_avg += loss_valid_tmp/len(sample_worker)
  server.aggregate_model(sample_worker)
  '''
  server.model.to(args.device)
  for worker in workers:
    acc_valid_tmp,loss_valid_tmp = test(server.model,args.criterion,worker.valloader)
    acc_valid_avg += acc_valid_tmp/len(workers)
    loss_valid_avg += loss_valid_tmp/len(workers)
  server.model.to('cpu')
  '''
  print('Epoch{}  loss:{}  accuracy:{}'.format(epoch+1,loss_valid_avg,acc_valid_avg))
  acc_train.append(acc_train_avg)
  loss_train.append(loss_train_avg)
  acc_valid.append(acc_valid_avg)
  loss_valid.append(loss_valid_avg)

  if early_stopping.validate(loss_valid_avg):
    print('Early Stop')
    break
    
end = time.time()

/home/acc12212vx/jupyter_env/lib/python3.6/site-packages/torch/nn/modules/rnn.py:582: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at  /pytorch/aten/src/ATen/native/cudnn/RNN.cpp:775.)
  self.dropout, self.training, self.bidirectional, self.batch_first)


Epoch1  loss:4.474835537539588  accuracy:3.8146841051639013
Epoch2  loss:4.320136395427915  accuracy:15.101252036950212
Epoch3  loss:4.112746657596694  accuracy:15.101252036950212
Epoch4  loss:3.8267593022849824  accuracy:15.10125203695021
Epoch5  loss:3.5776475098398004  accuracy:15.10125203695021
Epoch6  loss:3.438932335045602  accuracy:15.101252036950212
Epoch7  loss:3.3777567287286123  accuracy:15.10125203695021
Epoch8  loss:3.3496867037481732  accuracy:15.101252036950212
Epoch9  loss:3.3343932708104456  accuracy:15.10125203695021
Epoch10  loss:3.3254450533125137  accuracy:15.10125203695021
Epoch11  loss:3.319454373584853  accuracy:15.101252036950212
Epoch12  loss:3.3160468502177136  accuracy:15.101252036950214
Epoch13  loss:3.313062215513653  accuracy:15.10125203695021
Epoch14  loss:3.311489857236544  accuracy:15.10125203695021
Epoch15  loss:3.3094887998369  accuracy:15.101252036950209
Epoch16  loss:3.3073297040330045  accuracy:15.10125203695021
Epoch17  loss:3.3067724049091347  a

In [24]:
acc_test_personalized = []
loss_test_personalized = []

start = time.time()

for i,worker in enumerate(workers):
  acc_tmp,loss_tmp = worker.test()
  acc_test_personalized.append(acc_tmp)
  loss_test_personalized.append(loss_tmp)
  print('Worker{} accuracy:{}  loss:{}'.format(i+1,acc_tmp,loss_tmp))

end = time.time()

acc_test_personalized_avg = sum(acc_test_personalized)/len(acc_test_personalized)
loss_test_personalized_avg = sum(loss_test_personalized)/len(loss_test_personalized)
print('Test(personalized)  loss:{}  accuracy:{}'.format(loss_test_personalized_avg,acc_test_personalized_avg))

Worker1 accuracy:18.559882439382807  loss:3.0714070796966553
Worker2 accuracy:19.587628865979383  loss:3.2186944484710693
Worker3 accuracy:11.363636363636363  loss:3.0423784255981445
Worker4 accuracy:19.696969696969695  loss:3.0962367057800293
Worker5 accuracy:25.0  loss:2.8974359035491943
Worker6 accuracy:19.047619047619047  loss:3.162169933319092
Worker7 accuracy:20.0  loss:2.7484309673309326
Worker8 accuracy:20.66115702479339  loss:3.0136637687683105
Worker9 accuracy:33.333333333333336  loss:3.6699912548065186
Worker10 accuracy:16.129032258064516  loss:3.1718528270721436
Worker11 accuracy:24.299065420560748  loss:2.8420252799987793
Worker12 accuracy:17.5  loss:2.918381690979004
Worker13 accuracy:18.889745566692365  loss:3.064676841100057
Worker14 accuracy:15.868263473053892  loss:3.031698226928711
Worker15 accuracy:12.5  loss:3.0544371604919434
Worker16 accuracy:17.62669184765502  loss:3.092442512512207
Worker17 accuracy:19.23076923076923  loss:3.2892088890075684
Worker18 accuracy:2

In [26]:
acc_test_global = []
loss_test_global = []

start = time.time()

for i,worker in enumerate(workers):
  server.model = server.model.to(args.device)
  acc_tmp,loss_tmp = test(server.model,args.criterion,worker.testloader)
  acc_test_global.append(acc_tmp)
  loss_test_global.append(loss_tmp)
  print('Worker{} accuracy:{}  loss:{}'.format(i+1,acc_tmp,loss_tmp))

server.model = server.model.to('cpu')

end = time.time()

acc_test_global_avg = sum(acc_test_global)/len(acc_test_global)
loss_test_global_avg = sum(loss_test_global)/len(loss_test_global)
print('Test(global)  loss:{}  accuracy:{}'.format(loss_test_global_avg,acc_test_global_avg))

Worker1 accuracy:18.603967670830272  loss:3.0738987241472517
Worker2 accuracy:19.243986254295532  loss:3.2170419692993164
Worker3 accuracy:11.363636363636363  loss:3.045091390609741
Worker4 accuracy:19.696969696969695  loss:3.093329906463623
Worker5 accuracy:25.0  loss:2.889305591583252
Worker6 accuracy:19.047619047619047  loss:3.155130386352539
Worker7 accuracy:20.0  loss:2.7341465950012207
Worker8 accuracy:20.24793388429752  loss:3.0175766944885254
Worker9 accuracy:33.333333333333336  loss:3.6672255992889404
Worker10 accuracy:16.129032258064516  loss:3.1680707931518555
Worker11 accuracy:24.299065420560748  loss:2.8331663608551025
Worker12 accuracy:17.5  loss:2.9106369018554688
Worker13 accuracy:18.889745566692365  loss:3.0665109952290854
Worker14 accuracy:15.868263473053892  loss:3.03623628616333
Worker15 accuracy:12.5  loss:3.0432639122009277
Worker16 accuracy:17.469310670443814  loss:3.0892144441604614
Worker17 accuracy:19.23076923076923  loss:3.286276340484619
Worker18 accuracy:22

In [28]:
acc_tune_test_global = []
loss_tune_test_global = []

start = time.time()

for i,worker in enumerate(workers):
  worker.model = copy.deepcopy(server.model)
  worker.model = worker.model.to(args.device)
  _,_ = train(worker.model,args.criterion,worker.trainloader,args.local_epochs)
  acc_tmp,loss_tmp = test(worker.model,args.criterion,worker.testloader)
  acc_tune_test_global.append(acc_tmp)
  loss_tune_test_global.append(loss_tmp)
  print('Worker{} accuracy:{}  loss:{}'.format(i+1,acc_tmp,loss_tmp))
  worker.model = worker.model.to('cpu')
  del worker.model

end = time.time()

acc_tune_test_global_avg = sum(acc_tune_test_global)/len(acc_tune_test_global)
loss_tune_test_global_avg = sum(loss_tune_test_global)/len(loss_tune_test_global)
print('Test_fine-tune(global)  loss:{}  accuracy:{}'.format(loss_tune_test_global_avg,acc_tune_test_global_avg))

Worker1 accuracy:20.955180014695078  loss:3.017582586833409
Worker2 accuracy:18.213058419243985  loss:3.216718912124634
Worker3 accuracy:11.363636363636363  loss:3.046257972717285
Worker4 accuracy:19.696969696969695  loss:3.146597385406494
Worker5 accuracy:25.0  loss:2.893698215484619
Worker6 accuracy:19.047619047619047  loss:3.1696276664733887
Worker7 accuracy:20.0  loss:2.7699174880981445
Worker8 accuracy:20.24793388429752  loss:3.026095390319824
Worker9 accuracy:33.333333333333336  loss:3.664252519607544
Worker10 accuracy:15.725806451612904  loss:3.2126717567443848
Worker11 accuracy:24.299065420560748  loss:2.9204323291778564
Worker12 accuracy:17.5  loss:3.0243701934814453
Worker13 accuracy:19.35235158057055  loss:3.0517078240712485
Worker14 accuracy:15.868263473053892  loss:3.043837070465088
Worker15 accuracy:25.0  loss:3.040376663208008
Worker16 accuracy:17.469310670443814  loss:3.0914592146873474
Worker17 accuracy:17.94871794871795  loss:3.3046464920043945
Worker18 accuracy:22.97